<a href="https://colab.research.google.com/github/dlskawns/cp2/blob/main/wide_deep_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install category_encoders
!pip install kmodes

     |████████████████████████████████| 82 kB 430 kB/s 


In [ ]:

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2, l1_l2

from category_encoders import OrdinalEncoder
from kmodes.kprototypes import KPrototypes

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')
import pickle
import copy


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 정책데이터셋 클러스터링 및 유저데이터와의 결합 후 라벨 지정
with open('/content/drive/MyDrive/웰로/data/user_policy_clustered.pkl' , 'rb') as f:
  l = pickle.load(f)

In [ ]:
class recommendation_preprocessing:
    """
    추천 진행을 위한 클래스
    특정 유저 number 기입시, 해당 유저의 
    """


    def __init__(self):
      

      self.policy_data = pd.read_csv('/content/drive/MyDrive/웰로/data/policy_data.csv', encoding = 'utf-8')  # 정책데이터 룩업테이블
      self.policy_data.rename(columns = {i:i+'_y' if i not in ['정책ID', '서비스명', '대상연령시작','대상연령끝','소관기관유형','지원유형','자녀','농축수산인','보육지원(0~7세)', '교육지원(8~19세)'] else i for i in self.policy_data.columns }, inplace=True)
      # 특정유저 데이터 전처리
    def user_policy_preprocessing(self, user): 
      self.class_to_columns_general(user, 'mb_10+대상특성')
      self.income_change_user_general(user)
      user[['mb_15']].fillna('관심정책없음',inplace=True)
      self.class_to_columns_general(user, 'mb_15')
      user  = pd.DataFrame(user).T.drop(columns = ['시군구','자녀상세','자녀수','자녀','mb_10','mb_11','mb_10+대상특성', 'mb_11+관심상황특성','mb_12','mb_13','mb_14','mb_15'])
      user['결혼'] = user['결혼'].replace('미혼,기혼','무관')
      a = user.columns
      # 유저 정책 데이터 병합
      for i in a:
        self.policy_data[i] = user[i].values[0]
      self.policy_data = self.policy_data.rename(columns={'성별':'성별_x','학력':'학력_x','결혼':'결혼_x','농축수산인_x':' 농축수산인','교육지원(만8~19세)_x':'교육지원(만8~19세)','max_income':'max_income_x',
                                                  'min_income':'min_income_x','보육지원(만0~7세)_x':'보육지원(만0~7세)','관심정책없음_x':'관심정책없음','시도':'시도_x','직장':'직장_x'}) #'시도':'시도_x','직장':'직장_x'

      # self.policy_data = self.policy_data[cols]

      return self.policy_data
    def income_change_user_general(self, user):
      if user['mb_12'] != None:
        if '이하' in user['mb_12']:
          user['max_income'] = 40
          user['min_income'] = 0
        elif '이상' in user['mb_12']:
          user['max_income'] = 110     # 100이상은 모두 포함
          user['min_income'] = 100
        elif '기초생활수급자' in user['mb_12']:
          user['max_income'] = 30
          user['min_income'] = 0
        elif '사이' in user['mb_12']:
          if '40~60' in user['mb_12']:
            user['max_income'] = 60
            user['min_income'] = 40
          elif '60~80' in user['mb_12']:
            user['max_income'] = 80
            user['min_income'] = 60
          elif '80~100' in user['mb_12']:
            user['max_income'] = 100
            user['min_income'] = 80
          else: # 40% 이하
            user['max_income'] = 40
            user['min_income'] = 0
        else:
          user['max_income'] = 110     # 100이상은 모두 포함
          user['min_income'] = 0
      else:
        user['max_income'] = 110     
        user['min_income'] = 0


    def class_to_columns_general(self, user, col):
        """
        유저 추천을 위한 전처리 함수
        """
        b = []
        a = user[col]

        if col == 'mb_10+대상특성':
          mb_10_cols = ['다자녀가정', '농축수산인',	'장애인',	'질병/부상/질환자',	'북한이탈주민',	'한부모가정/조손가정',	'해당사항없음',	'국가유공자',	'다문화가족']
          if a != None:
            if ',' in a:
              a = a.split(',')
              b.extend(a)
            else:
              b.append(a) 
            for i in mb_10_cols:
              if i in b:
                user[i+'_x'] = 1
              else:
                user[i+'_x'] = 0
          elif a == None:
            for i in mb_10_cols:
              if i in '해당사항없음':
                user[i+'_x'] = 1
              else:
                user[i+'_x'] = 0
        elif col == 'mb_15':
          mb_15_cols = ['근로자 지원',	'교육지원(만8~19세)',	'취업 지원',	'문화생활 지원',	'의료 지원',	'보육지원(만0~7세)',	'관심정책없음',	'기업금융지원',	'주택-부동산 지원',	'개인금융지원',	'성인교육지원',	'창업 지원']
          if a != None:
            if ',' in a:
              a = a.split(',')
              b.extend(a)
            else:
              b.append(a) 
            for i in mb_15_cols:
              if i in b:
                user[i+'_x'] = 1
              else:
                user[i+'_x'] = 0
                
          elif a == None:
            for i in mb_15_cols:
              if i in '관심정책없음':
                user[i+'_x'] = 1
              else:
                user[i+'_x'] = 0
          user = pd.DataFrame(user).T.rename({'\t농축수산인_x': ' 농축수산인'})



In [ ]:
class Data:

    def __init__(self):
        self.utils = Utils()
        self.predict_features = None
        # self.categorical_columns = None
        self.crossed_columns_dic = None
    def get_wide_model_data(self, df_train, df_test= None, prediction = None):
        # x : 유저 / y: 정책
        # 중요 feature인 대상특성과 관심정책(mb_15) 관련 
        wide_cols = [ '성별_x', '결혼_x',
              ' 농축수산인', '해당사항없음_x', '한부모가정/조손가정_x', '국가유공자_x', '북한이탈주민_x',
              '질병/부상/질환자_x', '장애인_x', '다문화가족_x', '다자녀가정_x', '문화생활 지원_x', '주택-부동산 지원_x', '관심정책없음', '근로자 지원_x',
              '의료 지원_x', '보육지원(만0~7세)', '개인금융지원_x', '교육지원(만8~19세)', '성인교육지원_x',
              '기업금융지원_x', '취업 지원_x', '창업 지원_x',
              '성별_y', '결혼_y', '질병/부상/질환자_y', '국가유공자_y', '한부모가정/조손가정_y', '다자녀가정_y', '해당사항없음_y', '다문화가족_y', '북한이탈주민_y', '농축수산인', '장애인_y',
              '개인금융지원_y', '문화생활 지원_y', '취업 지원_y', '창업 지원_y', '교육지원(8~19세)','의료 지원_y', '보육지원(0~7세)', '주택-부동산 지원_y', '성인교육지원_y', '기업금융지원_y','근로자 지원_y']

        crossed_cols = (['성별_x', '성별_y'], ['결혼_x', '결혼_y']) #,['직장_x','직장_y'],['학력_x','학력_y'],['시도_x', '시도_y'])
        target = 'label'
       
        
        if prediction:
          df_wide = df_train
          for col_name, col_lst in self.crossed_columns_dic.items():
            df_wide[col_name] = df_wide[col_lst].apply(lambda x: '-'.join(x), axis=1)
          print('df_wide cols:',df_wide.columns)
          # dummy로 변경

          df_wide = pd.get_dummies(df_wide, columns=self.dummy_cols)

          d_lst = [x for x in df_wide.columns if x not in self.predict_features]
          df_wide = df_wide.drop(columns = d_lst)

          add_lst = [x for x in self.predict_features if x not in df_wide.columns]
          for i in add_lst:
            df_wide[i] = 0


          X_predict = np.array(df_wide[self.predict_features].values, dtype = np.float)

          return X_predict
        elif prediction == None:
          df_train['IS_TRAIN'] = 1
          df_test['IS_TRAIN'] = 0
          df_wide = pd.concat([df_train, df_test])

          categorical_columns = list(df_wide.select_dtypes(include=['object']).columns)
          self.crossed_columns_dic = self.utils.cross_columns(crossed_cols)
          wide_cols += list(self.crossed_columns_dic.keys())

          # crossed 변수 추가
          for col_name, col_lst in self.crossed_columns_dic.items():
              df_wide[col_name] = df_wide[col_lst].apply(lambda x: '-'.join(x), axis=1)
          df_wide = df_wide[wide_cols + [target] + ['IS_TRAIN']]

          # dummy로 변경
          self.dummy_cols = [wc for wc in wide_cols if wc in categorical_columns + list(self.crossed_columns_dic.keys())]
          df_wide = pd.get_dummies(df_wide, columns=self.dummy_cols)
          train = df_wide[df_wide.IS_TRAIN == 1].drop('IS_TRAIN', axis=1)
          test = df_wide[df_wide.IS_TRAIN == 0].drop('IS_TRAIN', axis=1)

          self.predict_features = [c for c in train.columns if c != target]

          X_train = np.array(train[self.predict_features].values, dtype = np.float)
          y_train = train['label'].values.reshape(-1, 1)

          X_test = np.array(test[self.predict_features].values, dtype = np.float)
          y_test = test['label'].values.reshape(-1, 1)
          return X_train, y_train, X_test, y_test

        
    def get_deep_model_data(self, df_train, df_test=None, prediction = None):

        embedding_cols = ['시도_x', '학력_x', '직장_x', '결혼_x',
              ' 농축수산인', '해당사항없음_x', '한부모가정/조손가정_x', '국가유공자_x', '북한이탈주민_x','질병/부상/질환자_x', '장애인_x', '다문화가족_x', '다자녀가정_x',
              '문화생활 지원_x', '주택-부동산 지원_x', '관심정책없음', '근로자 지원_x','의료 지원_x', '보육지원(만0~7세)', '개인금융지원_x', '교육지원(만8~19세)', '성인교육지원_x','기업금융지원_x', '취업 지원_x', '창업 지원_x',
              '시도_y', '소관기관유형', '지원유형', '학력_y', '성별_y', '결혼_y', '자녀', '직장_y',
              '농축수산인', '해당사항없음_y','한부모가정/조손가정_y', '국가유공자_y', '북한이탈주민_y', '질병/부상/질환자_y', '장애인_y', '다문화가족_y', '다자녀가정_y', 
              '개인금융지원_y', '문화생활 지원_y', '취업 지원_y', '창업 지원_y', '교육지원(8~19세)','의료 지원_y', '보육지원(0~7세)', '주택-부동산 지원_y', '성인교육지원_y', '기업금융지원_y','근로자 지원_y']
        cont_cols = ['나이','max_income_x','min_income_x','대상연령시작','대상연령끝','max_income_y', 'min_income_y'] 
        target = 'label'
        if prediction:
          df_deep = df_train
          deep_cols = embedding_cols + cont_cols
          df_deep = df_deep[deep_cols]


          scaler = StandardScaler()
          df_deep[cont_cols] = pd.DataFrame(scaler.fit_transform(df_train[cont_cols]), columns=cont_cols)
          df_deep, col_to_unique_val_num = self.utils.val2idx(df_deep, embedding_cols)

          embeddings_tensors = []
          for ec in embedding_cols:
              layer_name = ec + '_inp'
              inp = Input(shape=(1,), dtype='int64', name=layer_name)
              embd = Embedding(col_to_unique_val_num[ec], 8, input_length=1, embeddings_regularizer=l2(1e-3))(inp)
              embeddings_tensors.append((inp, embd))
              del (inp, embd)

          continuous_tensors = []
          for cc in cont_cols:
              layer_name = cc + '_in'
              inp = Input(shape=(1,), dtype='float32', name=layer_name)
              bulid = Reshape((1, 1))(inp)
              continuous_tensors.append((inp, bulid))
              del (inp, bulid)
          X = [df_deep[c] for c in deep_cols]
          return X, embeddings_tensors, continuous_tensors

        elif prediction == None:
          df_train['IS_TRAIN'] = 1
          df_test['IS_TRAIN'] = 0
          df_deep = pd.concat([df_train, df_test])
          deep_cols = embedding_cols + cont_cols
          df_deep = df_deep[deep_cols + [target, 'IS_TRAIN']]

          scaler = StandardScaler()
          df_deep[cont_cols] = pd.DataFrame(scaler.fit_transform(df_train[cont_cols]), columns=cont_cols)
          df_deep, col_to_unique_val_num = self.utils.val2idx(df_deep, embedding_cols)
  
          df_deep.fillna(-1, inplace=True)


          train = df_deep[df_deep.IS_TRAIN == 1].drop('IS_TRAIN', axis=1)
          test = df_deep[df_deep.IS_TRAIN == 0].drop('IS_TRAIN', axis=1)

          embeddings_tensors = []
          for ec in embedding_cols:
              layer_name = ec + '_inp'
              inp = Input(shape=(1,), dtype='int64', name=layer_name)
              embd = Embedding(col_to_unique_val_num[ec], 8, input_length=1, embeddings_regularizer=l2(1e-3))(inp)
              embeddings_tensors.append((inp, embd))
              del (inp, embd)

          continuous_tensors = []
          for cc in cont_cols:
              layer_name = cc + '_in'
              inp = Input(shape=(1,), dtype='float32', name=layer_name)
              bulid = Reshape((1, 1))(inp)
              continuous_tensors.append((inp, bulid))
              del (inp, bulid)

          X_train = [train[c] for c in deep_cols]
          y_train = np.array(train[target].values).reshape(-1, 1)

          X_test = [test[c] for c in deep_cols]
          y_test = np.array(test[target].values).reshape(-1, 1)

          return X_train, y_train, X_test, y_test, embeddings_tensors, continuous_tensors

class Utils:

    def cross_columns(self, x_cols):

        crossed_columns = dict()
        colnames = ['_'.join(x_c) for x_c in x_cols]
        for cname, x_c in zip(colnames, x_cols):
            crossed_columns[cname] = x_c
        return crossed_columns

    def val2idx(self, df, cols):

        val_types = dict()
        for c in cols:
            val_types[c] = df[c].unique()

        val_to_idx = dict()
        for k, v in val_types.items():
            val_to_idx[k] = {o: i for i, o in enumerate(val_types[k])}

        for k, v in val_to_idx.items():
            df[k] = df[k].apply(lambda x: v[x])

        unique_vals = dict()  # 사용한 값만
        for c in cols:
            unique_vals[c] = df[c].nunique()

        return df, unique_vals

In [ ]:

class Wide_Deep:

    def __init__(self, X_train_wide, y_train_wide,
                       X_train_deep, y_train_deep,
                       embeddings_tensors, continuous_tensors):
        # WIDE
        wide_input = Input(shape=(X_train_wide.shape[1],), dtype='float32', name='wide')
        # DEEP
        deep_input = [et[0] for et in embeddings_tensors] + [ct[0] for ct in continuous_tensors]
        deep_embedding = [et[1] for et in embeddings_tensors] + [ct[1] for ct in continuous_tensors]

        deep_embedding = Flatten()(concatenate(deep_embedding))
        layer_1 = Dense(50, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(deep_embedding)
        layer_1_dropout = Dropout(0.5)(layer_1)
        layer_2 = Dense(20, activation='relu', name='deep')(layer_1_dropout)
        layer_2_dropout = Dropout(0.5)(layer_2)

        # WIDE & DEEP
        wd_input = concatenate([wide_input, layer_2_dropout])
        wd_output = Dense(1, activation='sigmoid', name='wide_deep')(wd_input)

        self.wide_deep_model = Model([wide_input, deep_input], wd_output)
        self.wide_deep_model.compile(tf.keras.optimizers.Adam(learning_rate = 0.01, decay = 0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    def get_model(self):
        return self.wide_deep_model


class Run:

    def __init__(self):

        l_2 = l.copy()
        l_2.max_income_y = l_2.max_income_y.astype('int64')
        l_2.min_income_y = l_2.min_income_y.astype('int64')
        l_2.label = l_2.label.astype('int64')
        self.df_train, self.df_test = train_test_split(l_2, test_size = 0.2, random_state = 2)

        target = 'label'
        # self.model = None
        self.load = Data()


    def Wide_and_Deep(self):

        X_train_wide, y_train_wide, X_test_wide, y_test_wide = self.load.get_wide_model_data(self.df_train, self.df_test)
        X_train_deep, y_train_deep, X_test_deep, y_test_deep, \
        embeddings_tensors, continuous_tensors = self.load.get_deep_model_data(self.df_train, self.df_test)

        X_tr_wd = [X_train_wide] + X_train_deep
        y_tr_wd = y_train_deep

        X_te_wd = [X_test_wide] + X_test_deep
        y_te_wd = y_test_deep

        model = Wide_Deep(X_train_wide, y_train_wide, X_train_deep, y_train_deep, embeddings_tensors, continuous_tensors)
        early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3)
        self.model = model.get_model()
        self.model.fit(X_tr_wd, y_tr_wd, epochs=10, batch_size=128, validation_data = [X_te_wd, y_te_wd])

        print('wide and deep model accuracy:', self.model.evaluate(X_te_wd, y_te_wd)[1])
        
        eval_pred = self.model.predict(X_te_wd)
        eval_pred = eval_pred >= 0.5
        
        print(classification_report(y_te_wd, eval_pred))




    def Recommendation(self, user):
        rec = recommendation_preprocessing()
        user = rec.user_policy_preprocessing(user)
        
        x_wide =  self.load.get_wide_model_data(user, prediction=True)
        x_deep, embeddings_tensors, continuous_tensors = self.load.get_deep_model_data(user, prediction = True)
        x_input = [x_wide]+x_deep
        pred = self.model.predict(x_input)
        user['prob'] = pred
        user = user.sort_values(by = 'prob', ascending = False).reset_index()

        

        return user[['정책ID','서비스명','prob']].head(10)



In [ ]:
run = Run()
run.Wide_and_Deep()

Epoch 1/10
18750/18750 [==============================] - 184s 10ms/step - loss: 0.4793 - accuracy: 0.8713 - val_loss: 0.4127 - val_accuracy: 0.8731
Epoch 2/10
18750/18750 [==============================] - 182s 10ms/step - loss: 0.3896 - accuracy: 0.8730 - val_loss: 0.3740 - val_accuracy: 0.8736
Epoch 3/10
18750/18750 [==============================] - 183s 10ms/step - loss: 0.3641 - accuracy: 0.8730 - val_loss: 0.3583 - val_accuracy: 0.8727
Epoch 4/10
18750/18750 [==============================] - 182s 10ms/step - loss: 0.3516 - accuracy: 0.8731 - val_loss: 0.3478 - val_accuracy: 0.8741
Epoch 5/10
18750/18750 [==============================] - 184s 10ms/step - loss: 0.3441 - accuracy: 0.8733 - val_loss: 0.3422 - val_accuracy: 0.8732
Epoch 6/10
18750/18750 [==============================] - 183s 10ms/step - loss: 0.3391 - accuracy: 0.8734 - val_loss: 0.3372 - val_accuracy: 0.8718
Epoch 7/10
18750/18750 [==============================] - 186s 10ms/step - loss: 0.3356 - accuracy: 0.8732